<a href="https://colab.research.google.com/github/karasu1982/POS_Data_Analytics/blob/main/notebook/%E8%B3%BC%E8%B2%B7%E9%A0%86%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 購買順分析

購買順分析は、バスケット分析の応用で、商品Aを買う前に何を買っていたか、買った後に何を買ったかを分析する手法です。

用途として、ブランドのスイッチや継続といった観点で使われることが多いです。

1つ目は、同じカテゴリー内のブランドスイッチを分析するためです。

例えば、ブランドリニューアルで新しいシャンプーを出した場合に、どの程度の顧客が他社ブランドからスイッチしたか。これを分析することで、ブランドリニューアルの効果を測ることができます。

2つ目は、自社内のブランド継続を分析するためです。

例えば、セールでシャンプー本体が大幅に売り上げを伸ばしたとします。その後に、同じブランドのシャンプー詰め替えを使い続けてくれている顧客はどの程度いるのか。これを分析することで、セールの長期的な効果を測ることができます。

もちろん、同様の分析を他社ブランドで行うことで、自社が取られてしまった顧客について分析することも可能です。

## 環境設定

In [1]:
%%bash
pip install duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00


In [2]:
# 標準ライブラリ
import pandas as pd
import numpy as np

# データ見える化
import seaborn as sns
import matplotlib.pyplot as plt

# DuckDB
import duckdb

## データ準備

ID-POSのサンプルデータとして、下記を利用

https://www.kyoritsu-pub.co.jp/book/b10003634.html

In [ ]:
%%bash
wget https://kyoritsu-pub.sakura.ne.jp/app/file/goods_contents/2319.zip
unzip /content/2319.zip
unzip /content/DataProcess.20151001/src/data/Tafeng/Tafeng.zip

In [4]:
df = pd.read_csv("/content/Tafeng_dataset/Tafeng.csv")

# データ型を
df = df.astype({'CustID': 'object', 'ProductSubClass': 'object', 'ProductID': 'object'})

In [5]:
df.head(3)

,Time,CustID,Age,Area,ProductSubClass,ProductID,Amount,Asset,SalesPrice
0,2000-11-01 00:00:00,46855,D,E,110411,4710085120468,3,51,57
1,2000-11-01 00:00:00,539166,E,E,130315,4714981010038,2,56,48
2,2000-11-01 00:00:00,663373,F,E,110217,4710265847666,1,180,135


## データ集計

In [10]:
import duckdb

# SQLに慣れている方にとっては、少々くどいかもしれませんが、可読性を高めるために、処理を１つ１つ分解してWITH句で実行しています。
#
# 私がBigQueryの文法に慣れていることもあり、次の3点で修正が必要でした。他にも微妙な差はありそうなので、都度都度直す必要がありますね。
# ・コメントアウトは#は使わず、-- で行う
# ・文字列はダブルコーテーション（ " )ではなく、シングルクオーテーション（ ' )でくくる
# ・日付項目は、文字列を自動で日付型化しないため、DATETIME '2000-11-01'のように日付型を明示的に指定する
#

df_output = duckdb.query(
    f"""
    WITH
    t_all_before AS(
      -- 総来店者数（前期間）
      SELECT
       COUNT(DISTINCT CustID) AS Num_of_All_before
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
    ),

    t_all_after AS(
      -- 総来店者数（後期間）
      SELECT
       COUNT(DISTINCT CustID) AS Num_of_All_after
      FROM df
      WHERE Time BETWEEN DATETIME '2001-01-01' AND DATETIME '2001-02-28'
    ),

    t_purchaser_before AS(
      -- 商品ごとの購入者数（前期間）
      SELECT
        DISTINCT CAST(ProductID AS STRING) AS ProductID_A, CustID
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
    ),

    t_purchaser_after AS(
      -- 商品ごとの購入者数（後期間）
      SELECT
        DISTINCT CAST(ProductID AS STRING) AS ProductID_B, CustID
      FROM df
      WHERE Time BETWEEN DATETIME '2001-01-01' AND DATETIME '2001-02-28'
    ),


    t_base AS(
      -- 商品ごとの同時購入者
      SELECT
        ProductID_A, ProductID_B, COUNT(DISTINCT CustID) AS Num_of_Simultaneous_Purchaser
      FROM t_purchaser_before
      INNER JOIN t_purchaser_after
      USING(CustID)
      GROUP BY ProductID_A, ProductID_B
    )

    SELECT
      ProductID_A,
      Num_of_Purchaser_A,
      Num_of_Purchaser_A / Num_of_All_before AS PurchaseRate_A,
      ProductID_B,
      Num_of_Purchaser_B,
      Num_of_Purchaser_B / Num_of_All_after AS PurchaseRate_B,
      Num_of_Simultaneous_Purchaser,
      Num_of_Simultaneous_Purchaser / Num_of_Purchaser_A AS CombinedSalesRate,
      (Num_of_Simultaneous_Purchaser / Num_of_Purchaser_A) / (Num_of_Purchaser_B / Num_of_All_after) AS Lift

    FROM t_base

    LEFT OUTER JOIN (SELECT ProductID_A, COUNT(DISTINCT CustID) AS Num_of_Purchaser_A FROM t_purchaser_before GROUP BY ProductID_A)
    USING(ProductID_A)
    LEFT OUTER JOIN (SELECT ProductID_B, COUNT(DISTINCT CustID) AS Num_of_Purchaser_B FROM t_purchaser_after GROUP BY ProductID_B)
    USING(ProductID_B)


    FULL OUTER JOIN t_all_before
    ON True
    FULL OUTER JOIN t_all_after
    ON True
    """
).to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

作成したテーブルのアウトプット
*   ProductID_A ： 前期間に購買した商品の商品コード（商品A）
*   Num_of_Purchaser_A ： 商品Aの購入者数
*   PurchaseRate_A ： 商品Aの購入率（商品Aの購入者数÷前期間の総来店者数）
*   ProductID_B ： 後期間に購買した商品の商品コード（商品B）
*   Num_of_Purchaser_B ：商品Bの購入者数
*   PurchaseRate_B ： 商品Bの購入率（商品Bの購入者数÷後期間の総来店者数）
*   Num_of_Simultaneous_Purchaser：前期間に商品A、後期間に商品Bを購入した購入者数
*   CombinedSalesRate：前期間に商品A、後期間に商品Bを購入した購入者数÷前期間に商品Aの総購入者数
*   Lift：リフト値　（前期間に商品A、後期間に商品Bを購入した購入者数÷前期間に商品Aの総購入者数）÷（後期間に商品Bを購入した購入者数÷後期間の総来店者数）

前期間に商品A、後期間に商品Bを購入した購入者数とその率が多い順にソートします。すると、トップの商品の組み合わせ（商品A:4714981010038と商品B:4714981010038）は、順番に購入する率が0.376(37.6%)と、商品B全体の購入率16.3%よりも高くなっています。

リフト値は2.31となっており、商品Bを購入する確率よりも、商品A購入者があとにBを購入する確率の方が2.3倍高くなるといえます。

In [11]:
 # データ確認
df_output.sort_values(["Num_of_Simultaneous_Purchaser","CombinedSalesRate"], ascending = False)

,ProductID_A,Num_of_Purchaser_A,PurchaseRate_A,ProductID_B,Num_of_Purchaser_B,PurchaseRate_B,Num_of_Simultaneous_Purchaser,CombinedSalesRate,Lift
2016466,4714981010038,3333,0.141295,4714981010038,3946,0.163044,1253,0.375938,2.305738
5688425,4711271000014,4404,0.186697,4714981010038,3946,0.163044,1092,0.247956,1.520791
3927838,4711271000014,4404,0.186697,4719090900065,1735,0.071688,515,0.116939,1.631217
137255,4714981010038,3333,0.141295,4719090900065,1735,0.071688,481,0.144314,2.013082
3940213,4711271000014,4404,0.186697,4710265849066,1608,0.066441,432,0.098093,1.476392
...,...,...,...,...,...,...,...,...,...
7553879,4711271000014,4404,0.186697,4710644878793,3,0.000124,1,0.000227,1.831820
7555562,4711271000014,4404,0.186697,9789575665937,2,0.000083,1,0.000227,2.747729
7558346,4711271000014,4404,0.186697,4719857520321,6,0.000248,1,0.000227,0.915910
7559885,4711271000014,4404,0.186697,4710333741513,2,0.000083,1,0.000227,2.747729
